# Cholesky Decomposition

## Introduction

The **Cholesky decomposition** is a factorization of a Hermitian, positive-definite matrix into the product of a lower triangular matrix and its conjugate transpose. Named after André-Louis Cholesky, this decomposition is approximately twice as efficient as LU decomposition for solving systems of linear equations.

## Mathematical Foundation

### Definition

For a symmetric positive-definite matrix $\mathbf{A} \in \mathbb{R}^{n \times n}$, the Cholesky decomposition yields:

$$\mathbf{A} = \mathbf{L}\mathbf{L}^T$$

where $\mathbf{L}$ is a lower triangular matrix with positive diagonal entries.

### Positive Definiteness

A symmetric matrix $\mathbf{A}$ is **positive definite** if and only if:

$$\mathbf{x}^T \mathbf{A} \mathbf{x} > 0 \quad \forall \, \mathbf{x} \neq \mathbf{0}$$

Equivalently, all eigenvalues of $\mathbf{A}$ are strictly positive.

### Algorithm

The elements of $\mathbf{L}$ are computed as:

$$L_{jj} = \sqrt{A_{jj} - \sum_{k=1}^{j-1} L_{jk}^2}$$

$$L_{ij} = \frac{1}{L_{jj}} \left( A_{ij} - \sum_{k=1}^{j-1} L_{ik} L_{jk} \right) \quad \text{for } i > j$$

### Computational Complexity

The Cholesky decomposition requires $\frac{n^3}{3}$ floating-point operations, compared to $\frac{2n^3}{3}$ for LU decomposition.

## Applications

1. **Solving linear systems**: $\mathbf{A}\mathbf{x} = \mathbf{b}$ via forward and back substitution
2. **Monte Carlo simulations**: Generating correlated random variables
3. **Optimization**: Computing the inverse of covariance matrices
4. **Machine Learning**: Gaussian processes and Kalman filters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import cholesky, solve_triangular

# Set random seed for reproducibility
np.random.seed(42)

## Implementation

### Creating a Positive Definite Matrix

We construct a symmetric positive-definite matrix using $\mathbf{A} = \mathbf{B}^T\mathbf{B}$ where $\mathbf{B}$ is any matrix with full column rank.

In [ ]:
def create_positive_definite_matrix(n):
    """
    Create a symmetric positive-definite matrix of size n x n.
    
    Parameters
    ----------
    n : int
        Matrix dimension
        
    Returns
    -------
    A : ndarray
        Symmetric positive-definite matrix
    """
    B = np.random.randn(n, n)
    A = B.T @ B + n * np.eye(n)  # Add n*I to ensure strong positive definiteness
    return A

# Create a 5x5 positive definite matrix
n = 5
A = create_positive_definite_matrix(n)

print("Matrix A:")
print(np.round(A, 3))
print(f"\nEigenvalues of A: {np.round(np.linalg.eigvalsh(A), 3)}")
print("All eigenvalues are positive:", np.all(np.linalg.eigvalsh(A) > 0))

### Manual Cholesky Decomposition

We implement the Cholesky-Banachiewicz algorithm from scratch:

In [ ]:
def cholesky_decomposition(A):
    """
    Compute the Cholesky decomposition of a positive-definite matrix.
    
    Parameters
    ----------
    A : ndarray
        Symmetric positive-definite matrix
        
    Returns
    -------
    L : ndarray
        Lower triangular matrix such that A = L @ L.T
    """
    n = A.shape[0]
    L = np.zeros((n, n))
    
    for j in range(n):
        # Diagonal element
        sum_sq = sum(L[j, k]**2 for k in range(j))
        L[j, j] = np.sqrt(A[j, j] - sum_sq)
        
        # Off-diagonal elements
        for i in range(j + 1, n):
            sum_prod = sum(L[i, k] * L[j, k] for k in range(j))
            L[i, j] = (A[i, j] - sum_prod) / L[j, j]
    
    return L

# Compute Cholesky decomposition
L_manual = cholesky_decomposition(A)

print("Lower triangular matrix L (manual implementation):")
print(np.round(L_manual, 3))

### Verification

We verify our implementation against SciPy's optimized routine:

In [ ]:
# SciPy Cholesky (returns lower triangular when lower=True)
L_scipy = cholesky(A, lower=True)

print("Lower triangular matrix L (SciPy):")
print(np.round(L_scipy, 3))

# Verify reconstruction
A_reconstructed = L_manual @ L_manual.T
reconstruction_error = np.linalg.norm(A - A_reconstructed, 'fro')

print(f"\nReconstruction error ||A - LL^T||_F: {reconstruction_error:.2e}")
print(f"Manual vs SciPy difference: {np.linalg.norm(L_manual - L_scipy, 'fro'):.2e}")

## Solving Linear Systems

To solve $\mathbf{A}\mathbf{x} = \mathbf{b}$, we use the decomposition $\mathbf{A} = \mathbf{L}\mathbf{L}^T$:

1. Solve $\mathbf{L}\mathbf{y} = \mathbf{b}$ by forward substitution
2. Solve $\mathbf{L}^T\mathbf{x} = \mathbf{y}$ by back substitution

In [ ]:
def solve_cholesky(L, b):
    """
    Solve Ax = b using Cholesky decomposition.
    
    Parameters
    ----------
    L : ndarray
        Lower triangular Cholesky factor
    b : ndarray
        Right-hand side vector
        
    Returns
    -------
    x : ndarray
        Solution vector
    """
    # Forward substitution: Ly = b
    y = solve_triangular(L, b, lower=True)
    # Back substitution: L^T x = y
    x = solve_triangular(L.T, y, lower=False)
    return x

# Create a test problem
b = np.random.randn(n)
x_cholesky = solve_cholesky(L_scipy, b)

# Verify solution
residual = np.linalg.norm(A @ x_cholesky - b)
print(f"Solution x: {np.round(x_cholesky, 4)}")
print(f"Residual ||Ax - b||: {residual:.2e}")

## Performance Comparison

We compare the computational efficiency of Cholesky vs LU decomposition for solving linear systems:

In [ ]:
import time
from scipy.linalg import lu_factor, lu_solve

# Test different matrix sizes
sizes = [50, 100, 200, 500, 1000]
cholesky_times = []
lu_times = []

for size in sizes:
    # Create positive definite matrix
    A_test = create_positive_definite_matrix(size)
    b_test = np.random.randn(size)
    
    # Time Cholesky decomposition
    start = time.time()
    for _ in range(5):
        L_test = cholesky(A_test, lower=True)
        x_chol = solve_cholesky(L_test, b_test)
    cholesky_time = (time.time() - start) / 5
    cholesky_times.append(cholesky_time)
    
    # Time LU decomposition
    start = time.time()
    for _ in range(5):
        lu, piv = lu_factor(A_test)
        x_lu = lu_solve((lu, piv), b_test)
    lu_time = (time.time() - start) / 5
    lu_times.append(lu_time)
    
    print(f"n={size}: Cholesky={cholesky_time*1000:.2f}ms, LU={lu_time*1000:.2f}ms, "
          f"Speedup={lu_time/cholesky_time:.2f}x")

## Visualization

We visualize the matrix structure and performance comparison:

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Original matrix A
im1 = axes[0, 0].imshow(A, cmap='RdBu_r', aspect='equal')
axes[0, 0].set_title('Original Matrix $\\mathbf{A}$', fontsize=12)
axes[0, 0].set_xlabel('Column index')
axes[0, 0].set_ylabel('Row index')
plt.colorbar(im1, ax=axes[0, 0], fraction=0.046)

# Plot 2: Lower triangular factor L
im2 = axes[0, 1].imshow(L_scipy, cmap='RdBu_r', aspect='equal')
axes[0, 1].set_title('Cholesky Factor $\\mathbf{L}$', fontsize=12)
axes[0, 1].set_xlabel('Column index')
axes[0, 1].set_ylabel('Row index')
plt.colorbar(im2, ax=axes[0, 1], fraction=0.046)

# Plot 3: Sparsity pattern of L for a larger matrix
A_large = create_positive_definite_matrix(50)
L_large = cholesky(A_large, lower=True)
axes[1, 0].spy(L_large, markersize=2)
axes[1, 0].set_title('Sparsity Pattern of $\\mathbf{L}$ (n=50)', fontsize=12)
axes[1, 0].set_xlabel('Column index')
axes[1, 0].set_ylabel('Row index')

# Plot 4: Performance comparison
axes[1, 1].loglog(sizes, cholesky_times, 'o-', label='Cholesky', linewidth=2, markersize=8)
axes[1, 1].loglog(sizes, lu_times, 's-', label='LU', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Matrix size $n$', fontsize=11)
axes[1, 1].set_ylabel('Time (seconds)', fontsize=11)
axes[1, 1].set_title('Cholesky vs LU Decomposition Performance', fontsize=12)
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

# Add theoretical scaling lines
n_theory = np.array(sizes)
scale = cholesky_times[2] / (sizes[2]**3)
axes[1, 1].loglog(n_theory, scale * n_theory**3, '--', color='gray', 
                  alpha=0.5, label='$O(n^3)$')
axes[1, 1].legend(fontsize=10)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nFigure saved to plot.png")

## Numerical Stability Analysis

We examine the condition number and numerical stability:

In [ ]:
# Analyze condition number
cond_A = np.linalg.cond(A)
cond_L = np.linalg.cond(L_scipy)

print("Numerical Stability Analysis")
print("=" * 40)
print(f"Condition number of A: {cond_A:.2f}")
print(f"Condition number of L: {cond_L:.2f}")
print(f"κ(L)² ≈ κ(A): {cond_L**2:.2f} ≈ {cond_A:.2f}")

# Test with ill-conditioned matrix
print("\nIll-conditioned matrix test:")
ill_cond = np.diag([1, 1e-4, 1e-8, 1e-12, 1e-16])
ill_cond = ill_cond + 0.1 * np.random.randn(5, 5)
ill_cond = ill_cond.T @ ill_cond

try:
    L_ill = cholesky(ill_cond, lower=True)
    print(f"Cholesky succeeded, cond(A) = {np.linalg.cond(ill_cond):.2e}")
except np.linalg.LinAlgError as e:
    print(f"Cholesky failed: {e}")

## Application: Generating Correlated Random Variables

The Cholesky decomposition is essential for generating correlated random samples from a multivariate Gaussian distribution $\mathcal{N}(\boldsymbol{\mu}, \boldsymbol{\Sigma})$:

$$\mathbf{x} = \boldsymbol{\mu} + \mathbf{L}\mathbf{z}$$

where $\mathbf{z} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$ and $\boldsymbol{\Sigma} = \mathbf{L}\mathbf{L}^T$.

In [ ]:
# Define covariance matrix with known correlations
rho = 0.8  # Correlation coefficient
Sigma = np.array([[1.0, rho], 
                  [rho, 1.0]])
mu = np.array([0, 0])

# Cholesky decomposition
L_cov = cholesky(Sigma, lower=True)

# Generate correlated samples
n_samples = 1000
z = np.random.randn(2, n_samples)
samples = mu.reshape(-1, 1) + L_cov @ z

# Verify correlation
empirical_corr = np.corrcoef(samples[0], samples[1])[0, 1]
print(f"Target correlation: {rho}")
print(f"Empirical correlation: {empirical_corr:.4f}")

## Conclusion

The Cholesky decomposition provides:

1. **Computational efficiency**: Approximately 2× faster than LU for symmetric positive-definite systems
2. **Numerical stability**: Well-conditioned for positive-definite matrices
3. **Memory efficiency**: Only the lower triangular part needs storage
4. **Wide applicability**: Essential in statistics, optimization, and scientific computing

The relationship $\kappa(\mathbf{L})^2 \approx \kappa(\mathbf{A})$ implies that solving via Cholesky is as stable as the problem allows.